# Analisi AI vs Human

## import delle librerie

In [83]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [84]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import re

In [85]:
import seaborn as sns 

## caricamento del dataset 

In [86]:
# caricare il dataset 
df = pd.read_csv("ai_vs_human_dataset_medium.csv")
print("1. pulizia del dataset")
print(f"dataset caricato: {df.shape[0]} righe, {df.shape[1]} colonne")


1. pulizia del dataset
dataset caricato: 500 righe, 12 colonne


In [87]:
# panoramica dei dati 
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    int64  
 1   label             500 non-null    object 
 2   topic             500 non-null    object 
 3   text              500 non-null    object 
 4   length_chars      500 non-null    int64  
 5   length_words      500 non-null    int64  
 6   quality_score     500 non-null    float64
 7   sentiment         500 non-null    float64
 8   source_detail     500 non-null    object 
 9   timestamp         500 non-null    object 
 10  plagiarism_score  500 non-null    float64
 11  notes             157 non-null    object 
dtypes: float64(3), int64(3), object(6)
memory usage: 47.0+ KB


,id,length_chars,length_words,quality_score,sentiment,plagiarism_score
count,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000
mean,250.500000,115.152000,16.79800,3.502340,0.161240,0.124510
std,144.481833,39.208973,5.83449,0.900641,0.416834,0.095412
min,1.000000,65.000000,9.00000,1.500000,-0.590000,0.000000
25%,125.750000,87.000000,13.00000,2.847500,-0.170000,0.050000
50%,250.500000,103.000000,14.00000,3.540000,0.190000,0.104500
75%,375.250000,133.250000,20.00000,4.240000,0.490000,0.171500
max,500.000000,280.000000,41.00000,5.000000,0.900000,0.349000


## Cleaning

In [88]:
print(" 2. Pulizia del dataset mirata")

 2. Pulizia del dataset mirata


In [89]:
comment = """
Questa funzione serve a pulire e normalizzare il testo prima di qualsiasi analisi NLP.
Obiettivo principale: rimuovere elementi che non portano informazione stilistica o semantica 
ma potrebbero introdurre rumore o bias nel modello. In particolare:
1. Rimuove il carattere "|" usato come marker interno o separatore nei dati.
2. Elimina frasi boilerplate tipiche generate da AI (es. "Analysis indicates that…"), 
   che potrebbero creare un data leakage se il modello le imparasse come scorciatoia.
3. Normalizza spazi, newline e tabulazioni, riducendo il testo a un formato uniforme,
   pronto per tokenizzazione, feature extraction e modeling.
"""

print(comment)



Questa funzione serve a pulire e normalizzare il testo prima di qualsiasi analisi NLP.
Obiettivo principale: rimuovere elementi che non portano informazione stilistica o semantica 
ma potrebbero introdurre rumore o bias nel modello. In particolare:
1. Rimuove il carattere "|" usato come marker interno o separatore nei dati.
2. Elimina frasi boilerplate tipiche generate da AI (es. "Analysis indicates that…"), 
   che potrebbero creare un data leakage se il modello le imparasse come scorciatoia.
3. Normalizza spazi, newline e tabulazioni, riducendo il testo a un formato uniforme,
   pronto per tokenizzazione, feature extraction e modeling.



In [90]:
def clean_text_for_nlp(text):
    # 1. Rimozione marker di fonte interna e rumore tipico dell'AI
    text = re.sub(r'\|', '', text)
    # 2. Rimozione boilerplate (frasi standard che l'AI usa per iniziare)
    boilerplate_patterns = [
        r'^Analysis indicates that\s*', r'^The following summary on\s*', 
        r'^This article discusses\s*', r'^As someone who follows\s*', 
        r'^I recently experienced\s*', r'^In my experience,\s*',
        r'^Based on the data,\s*' # Aggiungiamo un pattern comune
    ]
    for pattern in boilerplate_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # 3. Normalizzazione: rimuove newline, tabulazioni e doppi spazi
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['text_cleaned'] = df['text'].apply(clean_text_for_nlp)
print(" - Colonna 'text' pulita in 'text_cleaned' (rimozione marker e boilerplate AI).")

 - Colonna 'text' pulita in 'text_cleaned' (rimozione marker e boilerplate AI).


In [91]:
comment = """
Questa funzione pulisce il testo rimuovendo simboli e caratteri indesiderati.
    In particolare:
    1. Rimuove tutte le emoji presenti nel testo usando il modulo `emoji`.
    2. Elimina qualsiasi carattere che non sia una lettera (a-z, A-Z), numero (0-9), 
       spazio o punteggiatura base (.,!?) tramite espressione regolare.
    3. Restituisce il testo "pulito", pronto per analisi NLP, tokenizzazione o feature extraction.
"""

print(comment)



Questa funzione pulisce il testo rimuovendo simboli e caratteri indesiderati.
    In particolare:
    1. Rimuove tutte le emoji presenti nel testo usando il modulo `emoji`.
    2. Elimina qualsiasi carattere che non sia una lettera (a-z, A-Z), numero (0-9), 
       spazio o punteggiatura base (.,!?) tramite espressione regolare.
    3. Restituisce il testo "pulito", pronto per analisi NLP, tokenizzazione o feature extraction.



In [92]:
import emoji 

def remove_special_chars(text):
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)
    return text

df['text_cleaned'] = df['text_cleaned'].apply(remove_special_chars)

In [93]:
comment = """
Questa sezione normalizza ulteriormente il testo e crea versioni standardizzate 
per l'analisi NLP. In particolare:

1. `text_cleaned`:
   - Mantiene le maiuscole originali.
   - Rimuove spazi iniziali e finali con `.str.strip()`.
   
2. `text_cleaned_lower`:
   - Converte tutto il testo in minuscolo usando `.str.lower()`.
   - Utile per NLP quando si vogliono confrontare parole senza distinzione tra maiuscole/minuscole.

3. Rimozione spazi multipli e newline:
   - `re.sub(r'\s+', '', x)` elimina spazi multipli e newline all'interno del testo.

4. Riduzione punteggiatura ripetuta:
   - `re.sub(r'([.,!?])\1+', r'\1', x)` sostituisce sequenze di punteggiatura ripetuta 
     (es. "!!!" o "..") con un singolo carattere.

Obiettivo generale: avere un testo **uniforme, pulito e coerente** pronto per 
tokenizzazione, feature extraction o modeling.
"""

print(comment)



Questa sezione normalizza ulteriormente il testo e crea versioni standardizzate 
per l'analisi NLP. In particolare:

1. `text_cleaned`:
   - Mantiene le maiuscole originali.
   - Rimuove spazi iniziali e finali con `.str.strip()`.

2. `text_cleaned_lower`:
   - Converte tutto il testo in minuscolo usando `.str.lower()`.
   - Utile per NLP quando si vogliono confrontare parole senza distinzione tra maiuscole/minuscole.

3. Rimozione spazi multipli e newline:
   - `re.sub(r'\s+', '', x)` elimina spazi multipli e newline all'interno del testo.

4. Riduzione punteggiatura ripetuta:
   - `re.sub(r'([.,!?])+', r'', x)` sostituisce sequenze di punteggiatura ripetuta 
     (es. "!!!" o "..") con un singolo carattere.

Obiettivo generale: avere un testo **uniforme, pulito e coerente** pronto per 
tokenizzazione, feature extraction o modeling.



<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Rosy\AppData\Local\Temp\ipykernel_10316\3672646691.py:1: SyntaxWarning: invalid escape sequence '\s'
  comment = """


In [94]:
# lowercase 
df['text_cleaned'] = df['text_cleaned'].str.strip() 
df['text_cleaned_lower'] = df['text_cleaned'].str.lower() 

print("text_cleaned - preserva maiuscole")
print("text_cleaned_lower - lowercase")

df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'\s+', '', x))
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'([.,!?])\1+', r'\1', x))

text_cleaned - preserva maiuscole
text_cleaned_lower - lowercase


## Pulizia mirata per pattern AI "sospetti"

In [95]:
comment = """
Feature Engineering: Rilevamento pattern tipici di testo generato da AI

1. `ai_patterns_detect`:
   - Lista di frasi e pattern linguistici tipici che compaiono frequentemente nei testi AI.
   - Esempi: "analysis indicates", "research suggests", "the following summary", ecc.

2. Funzione `count_ai_patterns(text)`:
   - Converte il testo in minuscolo (`text.lower()`) per confronti case-insensitive.
   - Conta quante volte ciascun pattern della lista appare nel testo.
   - Restituisce un numero intero: il totale dei pattern trovati.

3. Applicazione al DataFrame:
   - Crea la colonna `ai_pattern_count` che contiene, per ogni testo, il numero di pattern tipici AI.
   - Questa feature può essere utile per analisi esplorativa o come segnale di AI, 
     ma va usata con cautela nel modello finale per evitare **data leakage**.

4. Output informativo:
   - Stampa la media dei pattern AI nei testi etichettati come AI.
"""

print(comment)



Feature Engineering: Rilevamento pattern tipici di testo generato da AI

1. `ai_patterns_detect`:
   - Lista di frasi e pattern linguistici tipici che compaiono frequentemente nei testi AI.
   - Esempi: "analysis indicates", "research suggests", "the following summary", ecc.

2. Funzione `count_ai_patterns(text)`:
   - Converte il testo in minuscolo (`text.lower()`) per confronti case-insensitive.
   - Conta quante volte ciascun pattern della lista appare nel testo.
   - Restituisce un numero intero: il totale dei pattern trovati.

3. Applicazione al DataFrame:
   - Crea la colonna `ai_pattern_count` che contiene, per ogni testo, il numero di pattern tipici AI.
   - Questa feature può essere utile per analisi esplorativa o come segnale di AI, 
     ma va usata con cautela nel modello finale per evitare **data leakage**.

4. Output informativo:
   - Stampa la media dei pattern AI nei testi etichettati come AI.



In [96]:
# Feature Engineering: AI 
ai_patterns_detect = [
    'community response', 'research suggests', 'according to the data',
    'analysis indicates', 'this article discusses', 'the following summary'
]

def count_ai_patterns(text):
    text_lower = text.lower()
    return sum([1 for pattern in ai_patterns_detect if pattern in text_lower])
    
df['ai_pattern_count'] = df['text_cleaned'].apply(count_ai_patterns)
print(f" nella colonna ai_pattern_count AI avg: {df[df['label'] == 'ai']['ai_pattern_count'].mean():.2f}")


 nella colonna ai_pattern_count AI avg: 0.00


## Gestione dei valori mancanti mirata

In [97]:
print("3. Gestione dei valori mancanti")

3. Gestione dei valori mancanti


In [98]:
comment = """
Gestione della colonna 'notes' (variabile categorica)

1. Imputazione dei valori mancanti o vuoti:
   - Sostituisce le celle vuote (`''`) e i valori NaN con la stringa 'no_tone'.
   - Questo garantisce che tutti i record abbiano un valore valido nella colonna.

2. Conversione a tipo categorico:
   - Trasforma la colonna in tipo `category`, utile per:
     • Ridurre l’uso di memoria
     • Facilitare l’uso in modelli di machine learning che gestiscono feature categoriche

3. Output informativo:
   - Stampa un messaggio che conferma l’imputazione e la conversione a categoria.
"""


print(comment)



Gestione della colonna 'notes' (variabile categorica)

1. Imputazione dei valori mancanti o vuoti:
   - Sostituisce le celle vuote (`''`) e i valori NaN con la stringa 'no_tone'.
   - Questo garantisce che tutti i record abbiano un valore valido nella colonna.

2. Conversione a tipo categorico:
   - Trasforma la colonna in tipo `category`, utile per:
     • Ridurre l’uso di memoria
     • Facilitare l’uso in modelli di machine learning che gestiscono feature categoriche

3. Output informativo:
   - Stampa un messaggio che conferma l’imputazione e la conversione a categoria.



In [100]:
# 3.1 Colonna 'notes' (Categorica - Imputazione a 'no_tone')
df['notes'] = df['notes'].replace('', 'no_tone').fillna('no_tone')
df['notes'] = df['notes'].astype('category')
print(" - Imputazione Na/vuoti in 'notes' con 'no_tone' e conversione a categoria.")

 - Imputazione Na/vuoti in 'notes' con 'no_tone' e conversione a categoria.
